In [1]:
!pip install Biopython

     |████████████████████████████████| 2.3MB 3.4MB/s 


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from Bio import SeqIO

In [0]:
sequences = [s for s in SeqIO.parse("sequence.fasta", "fasta")]
metadata = pd.read_csv(r"https://raw.githubusercontent.com/samarth1107/Virus-stain_prediction/master/metadata.tsv",sep="\t",parse_dates=["Collection Date"])

In [4]:
metadata = metadata.sort_values(by=['Collection Date'], ascending=True)
metadata.head()

,Name,Sequence Accession,Complete Genome,Segment,Segment Length,Subtype,Collection Date,Host Species,Country,State/Province,Flu Season,Strain Name,Unnamed: 12
430,HA,MF535113,No,4,1701,H1N1,2009-01-01,IRD:Human,India,-N/A-,-N/A-,A/Jodhpur/1316/2009,NaN
125,HA,HM204566,No,4,1769,H1N1,2009-06-01,IRD:Human,India,-N/A-,-N/A-,A/Delhi/NIV57/2009,NaN
489,HA,HM204567,No,4,1709,H1N1,2009-06-01,IRD:Human,India,-N/A-,-N/A-,A/Mum/NIV261/2009,NaN
504,HA,CY075901,No,4,1760,H1N1,2009-06-21,IRD:Human,India,-N/A-,-N/A-,A/Pune/NIV161/2009,NaN
86,HA,CY075914,No,4,1756,H1N1,2009-06-27,IRD:Human,India,-N/A-,-N/A-,A/Che/NIV246/2009,NaN


In [0]:
metadata.iloc[542:,:]  # data for testing

In [0]:
metadata = metadata[metadata['Host Species'] == 'IRD:Human']

#Training data
training_metadata = metadata[metadata['Collection Date'] < datetime(2018, 1, 1)]
training_idxs = [i for i, s in enumerate(sequences) if s.id[3:11] in training_metadata['Sequence Accession'].values]
training_sequences = [sequences[i][30:1737] for i in training_idxs]

#Testing data
test_metadata = metadata[metadata['Collection Date'] >= datetime(2018, 1, 1)]
test_idxs = [i for i, s in enumerate(sequences) if s.id[3:11] in test_metadata['Sequence Accession'].values]
test_sequences = [sequences[i][30:1737] for i in test_idxs]

In [6]:
print(len(training_idxs))
print(len(test_idxs))

542
9


In [0]:
from collections import Counter
from copy import deepcopy
from sklearn.preprocessing import LabelBinarizer


def encode_array(sequences):

    #To find length of biggest sequence
    total_sequence = len(sequences)
    max_size_sequence = max(Counter([len(s) for s in sequences]).keys())

    #To find common acid that is presant in all amino acids
    common_amino_acid = set()
    for s in sequences:
        common_amino_acid = common_amino_acid.union(set(s))
    
    #To create one-hot encoding of amino acids that is common in all
    one_hot  = LabelBinarizer()
    one_hot.fit(list(common_amino_acid))
    total_common_amino_acid = len(common_amino_acid)

    #To convert sequence into array of character or amino_acids
    padded_sequences = deepcopy(sequences)

    #Padding
    for s in padded_sequences:
        while len(s) < max_size_sequence:s.seq+='-'

    #Sequence after padding
    seq_array = np.chararray(shape=(total_sequence,max_size_sequence),unicode=True)
    for i, seq in enumerate(padded_sequences):
        seq_array[i, :] = list(seq)

    encoded_array = np.zeros(shape=(total_sequence, max_size_sequence*total_common_amino_acid))

    for i in range(max_size_sequence):
        encoded_array[:,i*total_common_amino_acid:(i+1)*total_common_amino_acid] = one_hot.transform(seq_array[:, i])

    return encoded_array


In [0]:
sequence_array = encode_array(sequences)
training_array = sequence_array[training_idxs]
test_array = sequence_array[test_idxs]

In [25]:
print(sequence_array.shape)
print(training_array.shape)
print(test_array.shape)

(551, 10626)
(542, 10626)
(9, 10626)


In [12]:
!pip install --upgrade keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)


In [26]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Lambda, Dropout
from keras.models import Model, model_from_json
from keras import backend as K
from keras import objectives
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [27]:
# Set up VAE.
with tf.device('/gpu:0'):

    #parameters of model
    intermediate_dim = 1000
    encoding_dim = 3
    latent_dim = encoding_dim
    epsilon_std = 1.0
    nb_epoch = 250

    x = Input(shape=(training_array.shape[1],))
    z_mean = Dense(latent_dim)(x)
    z_log_var = Dense(latent_dim)(x)

    def sampling(args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(latent_dim, ), mean=0.,
                                  stddev=epsilon_std)
        return z_mean + K.exp(z_log_var / 2) * epsilon

    #loss function
    def vae_loss(x, x_decoded_mean):
        xent_loss = training_array.shape[1] * objectives.binary_crossentropy(x, x_decoded_mean)
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return xent_loss + kl_loss

    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
    x_decoded_mean = Dense(training_array.shape[1], activation='sigmoid')(z_mean)

    #variational autoencoder
    vae = Model(x, x_decoded_mean)
    vae.compile(optimizer='adam', loss=vae_loss)

    # build a model to project inputs on the latent space
    encoder = Model(x, z_mean)
    encoder_var = Model(x, z_log_var)

    #train test split
    x_train, x_test = train_test_split(training_array)

    early_stopping = EarlyStopping(monitor="val_loss", patience=2)


    # build the decoder
    encoded_input = Input(shape=(encoding_dim,))
    # retrieve the last layer of the autoencoder model
    decoder_layer = vae.layers[-1]
    # create the decoder model
    decoder = Model(inputs=encoded_input, outputs=decoder_layer(encoded_input))


    # Train the VAE to learn weights
    vae.fit(x_train, x_train,
            shuffle=True,
            epochs=nb_epoch,
            validation_data=(x_test, x_test),
            callbacks=[early_stopping],
           )

Train on 406 samples, validate on 136 samples
Epoch 1/250
406/406 [==============================] - 2s 4ms/step - loss: 7161.9321 - val_loss: 6268.2201
Epoch 2/250
406/406 [==============================] - 0s 184us/step - loss: 4718.6279 - val_loss: 3213.7021
Epoch 3/250
406/406 [==============================] - 0s 181us/step - loss: 2653.7828 - val_loss: 2040.7236
Epoch 4/250
406/406 [==============================] - 0s 192us/step - loss: 1876.9937 - val_loss: 1630.4732
Epoch 5/250
406/406 [==============================] - 0s 205us/step - loss: 1574.3136 - val_loss: 1426.5701
Epoch 6/250
406/406 [==============================] - 0s 189us/step - loss: 1409.7659 - val_loss: 1300.4684
Epoch 7/250
406/406 [==============================] - 0s 198us/step - loss: 1298.2581 - val_loss: 1206.3087
Epoch 8/250
406/406 [==============================] - 0s 194us/step - loss: 1216.9778 - val_loss: 1134.8979
Epoch 9/250
406/406 [==============================] - 0s 184us/step - loss: 1152.40

In [0]:
# Testing of encoder and decoder
test_embeddings_mean = encoder.predict(test_array)
test_binary_mean = decoder.predict(test_embeddings_mean)
for i in range(len(test_binary_mean)):
  test_binary_mean[i] = test_binary_mean[i].round()

In [0]:
def right_pad(sequences):
    padded_sequences = deepcopy(sequences)
    seq_lengths = compute_seq_lengths(sequences)

    for s in padded_sequences:
        while len(s) < max(seq_lengths.keys()):
            s.seq += '*'
    return padded_sequences


def compute_seq_lengths(sequences):
    seq_lengths = [len(s) for s in sequences]
    seq_lengths = Counter(seq_lengths)
    return seq_lengths


def seq2chararray(sequences):
    padded_sequences = right_pad(sequences)
    seq_lengths = compute_seq_lengths(sequences)
    char_array = np.chararray(shape=(len(sequences), max(seq_lengths.keys())),
                              unicode=True)
    for i, seq in enumerate(padded_sequences):
        char_array[i, :] = list(seq)
    return char_array


def compute_alphabet(sequences):
    alphabet = set()
    for s in sequences:
        alphabet = alphabet.union(set(s))

    return alphabet


def binary2chararray(sequences, binary_array):

    alphabet = compute_alphabet(sequences)
    seq_lengths = compute_seq_lengths(sequences)
    seq_array = seq2chararray(sequences)

    lb = LabelBinarizer()
    lb.fit(list(alphabet))

    char_array = np.chararray(shape=(len(binary_array),
                              max(seq_lengths.keys())), unicode=True)

    for i in range(seq_array.shape[1]):
        char_array[:, i] = lb.inverse_transform(binary_array[:, i*len(alphabet):(i+1)*len(alphabet)])

    return char_array

In [30]:
result=binary2chararray(sequences,test_binary_mean)
loop=-1
for seq in sequences[542:]:
  positive = 0
  negative = 0
  loop+=1
  for i in range(len(seq)):
    if (seq.seq)[i]==result[loop][i]:
      positive+=1
    else:
      negative+=1
  print(positive*100/len(seq))

99.47089947089947
98.88300999412111
99.05937683715462
98.41269841269842
99.23574368018812
99.3533215755438
97.03026841804683
99.00058788947678
98.77049180327869


In [0]:
x_train = encoder.predict(training_array)

In [0]:
x_train_lstm = x_train.reshape(542 ,1, 3)

In [0]:
x_train_lstm=[]
for i in range(len(x_train)-5):
  temp=[]
  for batch in range(i,i+5):
    temp.append(x_train[i])
  x_train_lstm.append(temp)
x_train_lstm=np.array(x_train_lstm)

In [88]:
x_train_lstm.shape

(537, 5, 3)

In [115]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding

model = Sequential()

model.add(LSTM(units=10, activation='relu', input_shape=(5,3)))
model.add(Dropout(0.2))

model.add(Dense(20, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(10, activation='softmax'))

opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 10)                560       
_________________________________________________________________
dropout_15 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 20)                220       
_________________________________________________________________
dropout_16 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                210       
Total params: 990
Trainable params: 990
Non-trainable params: 0
_________________________________________________________________


In [122]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

model.fit(x_train_lstm[0:],
          just[5:],
          epochs=3)

Epoch 1/3
537/537 [==============================] - 0s 897us/step - loss: nan - accuracy: 0.0000e+00
Epoch 2/3
537/537 [==============================] - 0s 305us/step - loss: nan - accuracy: 0.0000e+00
Epoch 3/3
537/537 [==============================] - 0s 316us/step - loss: nan - accuracy: 0.0000e+00


In [120]:
x_train_lstm[0:].shape

(537, 5, 3)

In [117]:
just = []
for i in x_train:
  just.append(np.sum(i))
just = np.array(just)
just[5:].shape

(537,)